In [4]:
import pandas as pd
import matplotlib.pylab as plt
from matplotlib import font_manager,rc
from selenium import webdriver
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import platform
import seaborn as sns
import sys
import time
import re

if platform.system()=='Windows':
    font_path='c:/Windows/Fonts/malgun.ttf'


In [5]:
### 지도 시각화

In [6]:
## 1. 데이터 준비 (1_crawling_raw.xlsx)
raw_total=pd.read_excel('data_instar/1_crawling_raw.xlsx')
raw_total.head()

# 위치정보 가져오기
location_counts=raw_total['place'].value_counts()
location_counts

Jeju                    271
Jeju-do                 179
Jungle Book by Alice    108
Seogwipo                 66
제주도 크리스마스 박물관            59
                       ... 
제주마당                      1
형제섬                       1
동복해녀식당                    1
종로 광장시장                   1
디스이즈핫                     1
Name: place, Length: 1028, dtype: int64

In [21]:
## 위치정보를 DataFrame 형태로 변환
import pandas as pd
location_counts_df=pd.DataFrame(location_counts)
location_counts_df

# 위치정보를 엑셀파일로 저장
location_counts_df.to_excel('data_instar/3_location_counts.xlsx')
location_counts_df

,place
Jeju,271
Jeju-do,179
Jungle Book by Alice,108
Seogwipo,66
제주도 크리스마스 박물관,59
...,...
제주마당,1
형제섬,1
동복해녀식당,1
종로 광장시장,1


In [22]:
## 위치(지명)만 별도로 종류 확인
locations=list(location_counts_df.index)
locations

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 'Seogwipo',
 '제주도 크리스마스 박물관',
 '할로비치',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 '1100고지',
 'Nimome',
 '월정리해변',
 '고집돌우럭중문점',
 '석부작박물관',
 '제주도 애월읍',
 '알뜨르 비행장',
 '카멜리아 힐',
 '제주레포츠랜드',
 '밥깡패',
 '제주 함덕 서우봉 해변',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '위미동백나무군락지',
 '제주신화월드 Jeju Shinhwa World',
 'Woljeongri Beach',
 '협재해변 Beach',
 '연동 바오젠 거리',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '고래배꼽',
 '제주 성산 신산포구자연산횟집 회포장',
 '새별오름',
 '용머리해안',
 '용눈이오름',
 '캔디원',
 '사려니숲길',
 '하이엔드 제주',
 '제주빅볼랜드',
 '서귀포 느영나영 게스트하우스',
 '휴애리 자연생활공원',
 '옹포리',
 '삼무공원',
 '마마뜰',
 '제주어린왕자게스트하우스',
 '위미동백나무군락',
 '제주커피박물관 Baum',
 'Jeju Aewol',
 '한라산',
 '위미2리 동백군락지',
 'Jeju Island 제주특별자치도 济州道',
 '광치기해변',
 '서우봉',
 '제주해남',
 '한라산 (漢拏山, Hallasan)',
 '카페한라산',
 '성이시돌목장',
 '제주 송악산',
 '옹포별장가든',
 '월정리카페콧수염',
 '牛岛 Udo Island 우도',
 '곽지해수욕장',
 '카페브리프',
 '아줄레주',
 '에코랜드',
 '산굼부리',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 'Terarosa - 테라로사',

In [9]:
## KAKAO API 사용 (로컬/지도 API) : 장소 검색
import requests
searching='스타벅스 합정점'
url=f'http://dapi.kakao.com/v2/local/search/keyword.json?query={searching}'
header={
    "Authorization": "KakaoAK f607c1d8c4b841b697e7a13c44562dee"
    # 입력시 반드시 kakaoak 뒤에 한칸 띄고 api키를 적어야 한다.
}
place=requests.get(url,headers=header).json()['documents']
place

[{'address_name': '서울 마포구 합정동 472',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '2057327896',
  'phone': '1522-3232',
  'place_name': '스타벅스 합정점',
  'place_url': 'http://place.map.kakao.com/2057327896',
  'road_address_name': '서울 마포구 월드컵로1길 14',
  'x': '126.91253700818196',
  'y': '37.54994959743763'}]

In [15]:
## 카카오 로컬 API를 활용한 장소 검색 함수 만들기..
# 인자 : searching
# place[첫번째], name-장소이름, x,y  반환값 
# 반환값 : [name,x,y,searching]

def  find_places(searching):
    url=f'http://dapi.kakao.com/v2/local/search/keyword.json?query={searching}'
    header={
        "Authorization":'KakaoAK f607c1d8c4b841b697e7a13c44562dee'
    }
    places=requests.get(url,headers=header).json()['documents']
    place=places[0]
    name=place['place_name']
    x=place['x']
    y=place['y']
    data=[name,x,y,searching]
    return data


In [16]:
## 함수 확인
data=find_places('제주공항')
data

['제주국제공항', '126.492769004244', '33.5070789578184', '제주공항']

In [12]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tqdm in c:\users\djagk\appdata\local\programs\python\python310\lib\site-packages (4.64.0)



You should consider upgrading via the 'c:\Users\djagk\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [13]:
from tqdm.notebook import tqdm

In [ ]:
# import time
location_inform=[]
for location in tqdm(locations):
    try:
        data=find_places(location)
        location_inform.append(data)
        time.sleep(0.5)
    except:
        pass
location_inform

In [18]:
## 생성된 정보로 DF로 만들어서 Excel로 지정
locations_inform_df=pd.DataFrame(location_inform)
locations_inform_df.columns=['name_official','경도','위도','인스타위치명']
locations_inform_df.to_excel('data_instar/3_locations.xlsx',index=False)


In [38]:
#인스타 게시랑 파일(3_3_locaction_counts.xlsx), 위치정보 데이터 불러오기
location_counts_df=pd.read_excel('data_instar/3_location_counts.xlsx',index_col=0)
location_inform_df=pd.read_excel('data_instar/3_locations.xlsx')
#print(location_counts_df)
# 병합하기 (위치 데이터 병합)
location_data=pd.merge(location_inform_df,location_counts_df,how='inner',left_on='name_official',right_index=True)
location_data

,name_official,경도,위도,인스타위치명,place
1,서귀포잠수함,126.558616,33.239303,Seogwipo,1
620,서귀포잠수함,126.558616,33.239303,서귀포잠수함,1
4,제주에인감귤밭,126.539014,33.256561,제주에인감귤밭,48
5,제주도,126.545874,33.379778,Jeju Island,4
84,제주도,126.545874,33.379778,제주도,4
...,...,...,...,...,...
791,하림각,126.962725,37.597080,하림각,1
793,탕196,126.895852,37.627179,탕196,1
794,제주마당,126.458174,33.498104,제주마당,1
795,형제섬,128.644666,34.710915,형제섬,1


In [39]:
## 데이터 중복 점검하기
location_data['name_official'].value_counts()

제주동문재래시장    4
오설록티뮤지엄     3
자매국수 본점     3
서귀포잠수함      2
빌라드아토       2
           ..
쉐프부부        1
이게밥도둑       1
숨비아일랜드      1
삼공주회센터      1
디스이즈핫       1
Name: name_official, Length: 376, dtype: int64

In [49]:
## 장소이름을 기준으로 병합(pivot_table)
location_data=location_data.pivot_table(index=['name_official','경도','위도'],values='place',aggfunc='sum')
location_data


,,,place
name_official,경도,위도,
73st,126.455109,33.456989,1
가드망제,126.532954,33.500991,1
가배,126.951868,37.738173,1
가시식당,126.771438,33.353060,2
가시아방국수,126.918065,33.438605,2
...,...,...,...
황궁쟁반짜장,127.394311,36.338770,1
황우지해안,126.550528,33.241467,2
훈남횟집,126.667858,33.542086,1


In [50]:
## 병합 데이터 저장
location_data.to_excel('data_instar/3_location_inform.xlsx')

In [52]:
## 위에서 사용한 location_data('data_instar/3_location_inform.xlsx')
location_data=pd.read_excel('data_instar/3_location_inform.xlsx')
location_data.info()
location_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_official  376 non-null    object 
 1   경도             377 non-null    float64
 2   위도             377 non-null    float64
 3   place          377 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 11.9+ KB


,name_official,경도,위도,place
0,73st,126.455109,33.456989,1
1,가드망제,126.532954,33.500991,1
2,가배,126.951868,37.738173,1
3,가시식당,126.771438,33.353060,2
4,가시아방국수,126.918065,33.438605,2


In [53]:
## folium을 이용한 지도 시각화
#pip install folium


SyntaxError: invalid syntax (3813623116.py, line 2)

In [ ]:
pip install folium

In [69]:
import folium
MT_Hanla=[33.362500,126.533694]
map_jeju=folium.Map(location=MT_Hanla,zoom_start=11)    # 줌 크기 조절
for i in range(len(location_data)):
    name=location_data['name_official'][i]  #공식 명칭
    count=location_data['place'][i]
    size=int(count)*2 
    long=float(location_data['위도'][i])
    lat=float(location_data['경도'][i])
    folium.CircleMarker((long,lat),radius=size,color='red',popup=name).add_to(map_jeju)

map_jeju

In [70]:
map_jeju.save('data_instar/3_jeju.html')

In [78]:
## 마커(동그라미) 집합을 통한 지도 표시
from folium.plugins import MarkerCluster
locations=[]
names=[]

for i in range(len(location_data)):
    data=location_data.iloc[i] # 행 하나씩
    locations.append((float(data['위도']),float(data['경도']))) #위도 경도순으로 저장
    names.append(data['name_official'])

MT_Hanla=[33.362500,126.533694]
map_jeju2=folium.Map(location=MT_Hanla, zoom_start=11)
marker_cluster=MarkerCluster(
    locations=locations,popup=names,
    name='Jeju',
    overlay=True,
    control=True
)
marker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)
map_jeju2

In [79]:
map_jeju2.save('data_instar/3_jeju_cluster.html')

In [80]:
## 특정 단어 포함 게시글 찾기
#데이터 준비
import pandas as pd
raw_total=pd.read_excel('data_instar/1_crawling_raw.xlsx')
raw_total.head()

,content,date,like,place,tags
0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [111]:
# 단어 선택하기

# 예시 해돋이
select_word='해돋이'

check_list=[]

for content in raw_total['content']:
    if select_word in content:
        check_list.append(True)
    else:
        check_list.append(False)
raw_total[check_list]
#select_df.head()


,content,date,like,place,tags
34,..“쿠룽쿠룽 해트맨 마씸!”.제주방언으로 ‘쿨쿨 자던 해가 떴습니다’.다사다난 했...,2019-01-01,89,서귀포 법환포구,[]
73,아듀 2018!🤧👋🏻2019🐷황금돼지해의 첫 일출🌅을 만나볼 수 있는 제주 일출명소...,2018-12-31,18,Jeju-do,"['#제주', '#제주도', '#서귀포', '#제주도민', '#제주여행', '#제주..."
142,4일간 휴 ~ 뭐하지? ㅠ 뭐할까?. #해넘이#해돋이 어디로가지? .누워서보는극장?...,2018-12-28,94,NaN,"['#해넘이', '#해돋이', '#제주', '#jejudo', '#jeju', '#..."
441,순간포착엄청 놀랐네요갑자기 새들이 무더기로 나타나서날아오르길래 ㆍ그쪽에 무슨일 있는...,2018-12-18,77,제주프레임스파빌,"['#제주도공항', '#프레임펜션', '#프레임스파빌', '#제주', '#제주프레임..."
2458,해돋이#제주 #혼저옵서예 #제주스타그램 #제주도 #제주여행 #제주관광 #제주도여행 ...,2018-10-23,81,NaN,"['#제주', '#혼저옵서예', '#제주스타그램', '#제주도', '#제주여행', ..."
3943,곽지스테이션안녕 반가워..2019년...붉은해의 기운을 받으세요. 얍얍얍!!!..#...,2019-01-01,31,곽지스테이션,"['#곽지스테이션', '#곽지해수욕장', '#해돋이', '#2019', '#기해년'..."
4635,제주에서 마지막 밥은...해물라면구름낀 날씨에 전국에서 제주빼고 다 보였다는 해돋이...,2019-01-01,13,NaN,"['#제주도', '#제주도맛집', '#성산일출봉맛집', '#제주경미네', '#경미네..."
4715,#제주 를 제외한 전국에서 해돋이를 볼수 있....에서의 그 제주.... 포기하고 ...,2019-01-01,45,광치기해변,"['#제주', '#호텔', '#빼꼼']"
5112,사장님 아침 대바람부터 문열어 주셔서 감사해요 ㅋㅋㅋㅋ 참 친절하셔요 또 방문 하고...,2019-01-01,114,Woljeongri Beach,"['#월정리', '#월정리해변', '#월정리맛집', '#밥세끼', '#제주도맛집',..."
5248,#어부피자 #2018년 찾아주셔서 #감사합니다#2019년도 더 더욱 #노력하는 어부...,2018-12-31,35,NaN,"['#어부피자', '#2018년', '#감사합니다', '#2019년도', '#노력하..."


In [ ]:
## 선택한 데이터 확인
for i in select_df.index:
    print(select_df.loc[i,'content'])
    print('-'*50)


In [95]:
fpath=f'data_instar/4_select_data_{select_word}.xlsx'
 
select_df.to_excel(fpath,index=False)

In [117]:
## 여러개의 단어 선택/추출/저장하세요
# 선택 단어 리스트: '해돋이','박물관','힐링','게스트하우스','섭지코지'
# 각 선택단어를 추출하여 저장 (4_select_data_{선택단어}.xlsx)로 저장
ch_list=[]
words=['해돋이','박물관','힐링','게스트하우스','섭지코지']
raw_total=pd.read_excel('data_instar/1_crawling_raw.xlsx')

for word in words:
    ch_list=[]
    for content in raw_total['content']:
        if word in content:
            ch_list.append(True)
        else:
            ch_list.append(False)
    select_df=raw_total[ch_list]
    fpath=f'data_instar/4_select_data_{word}.xlsx'
    select_df.to_excel(fpath)

    